In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')


## 2. Load and check data
### 2.1 Load data

In [28]:
# Load data
##### Load train and Test set

train = pd.read_csv("C:/Users/adoko/PycharmProjects/pythonProject1/inputs/titanic_train.csv")
test = pd.read_csv("C:/Users/adoko/PycharmProjects/pythonProject1/inputs/titanic_test.csv")
IDtest = test["PassengerId"]

### 2.2 Outlier detection

In [29]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from Age, SibSp , Parch and Fare
Outliers_to_drop = detect_outliers(train,2,["Age","SibSp","Parch","Fare"])
# Drop outliers
train = train.drop(Outliers_to_drop, axis = 0).reset_index(drop=True)


dataset_2 =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)

### 2.3 joining train and test set

In [30]:
## Join train and test datasets in order to obtain the same number of features during categorical conversion
train_len = len(train)
dataset =  pd.concat(objs=[train, test], axis=0).reset_index(drop=True)
# Fill empty and NaNs values with NaN
dataset = dataset.fillna(np.nan)

## 3. Feature analysis
### 3.1 Numerical values

In [31]:
def feature_analysis(dataset,train):
    #Fill Fare missing values with the median value
    dataset["Fare"] = dataset["Fare"].fillna(dataset["Fare"].median())
    # Apply log to Fare to reduce skewness distribution
    dataset["Fare"] = dataset["Fare"].map(lambda i: np.log(i) if i > 0 else 0)
    train[["Sex", "Survived"]].groupby('Sex').mean()
    #Fill Embarked nan values of dataset set with 'S' most frequent value
    dataset["Embarked"] = dataset["Embarked"].fillna("S")
    # convert Sex into categorical value 0 for male and 1 for female
    dataset["Sex"] = dataset["Sex"].map({"male": 0, "female": 1})
    index_NaN_age = list(dataset["Age"][dataset["Age"].isnull()].index)
    for i in index_NaN_age:
        age_med = dataset["Age"].median()
        age_pred = dataset["Age"][(
                    (dataset['SibSp'] == dataset.iloc[i]["SibSp"]) & (dataset['Parch'] == dataset.iloc[i]["Parch"]) & (
                        dataset['Pclass'] == dataset.iloc[i]["Pclass"]))].median()
        if not np.isnan(age_pred):
            dataset['Age'].iloc[i] = age_pred
        else:
            dataset['Age'].iloc[i] = age_med
    # Get Title from Name
    dataset_title = [i.split(",")[1].split(".")[0].strip() for i in dataset["Name"]]
    dataset["Title"] = pd.Series(dataset_title)
    # Convert to categorical values Title
    dataset["Title"] = dataset["Title"].replace(
        ['Lady', 'the Countess', 'Countess', 'Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],
        'Rare')
    dataset["Title"] = dataset["Title"].map(
        {"Master": 0, "Miss": 1, "Ms": 1, "Mme": 1, "Mlle": 1, "Mrs": 1, "Mr": 2, "Rare": 3})
    dataset["Title"] = dataset["Title"].astype(int)
    # Drop Name variable
    dataset.drop(labels=["Name"], axis=1, inplace=True)
    # Create a family size descriptor from SibSp and Parch
    dataset["Fsize"] = dataset["SibSp"] + dataset["Parch"] + 1
    # Create new feature of family size
    dataset['Single'] = dataset['Fsize'].map(lambda s: 1 if s == 1 else 0)
    dataset['SmallF'] = dataset['Fsize'].map(lambda s: 1 if s == 2 else 0)
    dataset['MedF'] = dataset['Fsize'].map(lambda s: 1 if 3 <= s <= 4 else 0)
    dataset['LargeF'] = dataset['Fsize'].map(lambda s: 1 if s >= 5 else 0)
    # convert to indicator values Title and Embarked
    dataset = pd.get_dummies(dataset, columns=["Title"])
    dataset = pd.get_dummies(dataset, columns=["Embarked"], prefix="Em")
    # Replace the Cabin number by the type of cabin 'X' if not
    dataset["Cabin"] = pd.Series([i[0] if not pd.isnull(i) else 'X' for i in dataset['Cabin']])
    dataset = pd.get_dummies(dataset, columns=["Cabin"], prefix="Cabin")
    ## Treat Ticket by extracting the ticket prefix. When there is no prefix it returns X.
    Ticket = []
    for i in list(dataset.Ticket):
        if not i.isdigit():
            Ticket.append(i.replace(".", "").replace("/", "").strip().split(' ')[0])  #Take prefix
        else:
            Ticket.append("X")
    dataset["Ticket"] = Ticket
    dataset = pd.get_dummies(dataset, columns=["Ticket"], prefix="T")
    # Create categorical values for Pclass
    dataset["Pclass"] = dataset["Pclass"].astype("category")
    dataset = pd.get_dummies(dataset, columns=["Pclass"], prefix="Pc")
    # Drop useless variables
    dataset.drop(labels=["PassengerId"], axis=1, inplace=True)
    return dataset

dataset = feature_analysis(dataset,train)

C:\Users\adoko\AppData\Local\Temp\ipykernel_11976\2537478883.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Age'].iloc[i] = age_pred
C:\Users\adoko\AppData\Local\Temp\ipykernel_11976\2537478883.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Age'].iloc[i] = age_pred
C:\Users\adoko\AppData\Local\Temp\ipykernel_11976\2537478883.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['Age'].iloc[i] = age_

## 6. MODELING

In [13]:
train = dataset_2[:train_len]
test = dataset_2[train_len:]
test.drop(labels=["Survived"],axis = 1,inplace=True)
## Separate train dataset and test dataset


C:\Users\adoko\AppData\Local\Temp\ipykernel_11976\3000895439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(labels=["Survived"],axis = 1,inplace=True)


### 6.1 Simple modeling
#### 6.1.1 Cross validate models

I compared 10 popular classifiers and evaluate the mean accuracy of each of them by a stratified kfold cross validation procedure.

* SVC
* Decision Tree
* AdaBoost 
* Random Forest
* Extra Trees
* Gradient Boosting
* Multiple layer perceprton (neural network)
* KNN
* Logistic regression
* Linear Discriminant Analysis

In [14]:
# Cross validate model with Kfold stratified cross val
train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=["Survived"],axis = 1,inplace=True)

train["Survived"] = train["Survived"].astype(int)
Y_train = train["Survived"]
X_train = train.drop(labels = ["Survived"],axis = 1)


kfold = StratifiedKFold(n_splits=3)
folds = kfold.split(X_train,Y_train)
print(folds)


<generator object _BaseKFold.split at 0x000001BFD2316650>


C:\Users\adoko\AppData\Local\Temp\ipykernel_11976\3823796415.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(labels=["Survived"],axis = 1,inplace=True)
C:\Users\adoko\AppData\Local\Temp\ipykernel_11976\3823796415.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Survived"] = train["Survived"].astype(int)


In [22]:
# Cross validate model with Kfold stratified cross val


# Modeling step Test differents algorithms
random_state = 2
classifiers = []
classifiers.append(SVC(random_state=random_state, probability=True))
classifiers.append(DecisionTreeClassifier(random_state=random_state))
classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=random_state),random_state=random_state,learning_rate=0.1))
classifiers.append(RandomForestClassifier(random_state=random_state))
classifiers.append(ExtraTreesClassifier(random_state=random_state))
classifiers.append(GradientBoostingClassifier(random_state=random_state))
classifiers.append(MLPClassifier(random_state=random_state))
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression(random_state = random_state))
classifiers.append(LinearDiscriminantAnalysis())

cv_results = {}
for classifier in classifiers :
    cv_results[classifier]=cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4).mean()

keys = list(cv_results.keys())

{SVC(probability=True, random_state=2): 0.6832942505785021, DecisionTreeClassifier(random_state=2): 0.7661961257768182, AdaBoostClassifier(estimator=DecisionTreeClassifier(random_state=2),
                   learning_rate=0.1, random_state=2): 0.7968393273122674, RandomForestClassifier(random_state=2): 0.809303243481693, ExtraTreesClassifier(random_state=2): 0.782069141649834, GradientBoostingClassifier(random_state=2): 0.825222694310944, MLPClassifier(random_state=2): 0.82181746418704, KNeighborsClassifier(): 0.7627831564935418, LogisticRegression(random_state=2): 0.8161407907873047, LinearDiscriminantAnalysis(): 0.8149992647798596}


I decided to choose the SVC, AdaBoost, RandomForest , ExtraTrees and the GradientBoosting classifiers for the ensemble modeling.

#### 6.1.2 Hyperparameter tunning for best models

I performed a grid search optimization for AdaBoost, ExtraTrees , RandomForest, GradientBoosting and SVC classifiers.

I set the "n_jobs" parameter to 4 since i have 4 cpu . The computation time is clearly reduced.

But be carefull, this step can take a long time, i took me 15 min in total on 4 cpu.

### 6.2 Ensemble modeling
#### 6.2.1 Combining models

I choosed a voting classifier to combine the predictions coming from the 5 classifiers.

I preferred to pass the argument "soft" to the voting parameter to take into account the probability of each vote.

In [25]:
votingC = VotingClassifier(estimators=[('rfc', keys[0]), ('extc', keys[1]),
('svc', keys[4]), ('adac',keys[3]),('gbc',keys[2]), ('g123bc',keys[6]),('gb123c2',keys[7]),('gbc34',keys[8]),('gbc2',keys[9])], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train, Y_train)
votingC.score(X_train,Y_train)
print(votingC.score(X_train,Y_train))

0.9727582292849035


### 6.3 Prediction
#### 6.3.1 Predict and Submit results

In [30]:
test_Survived = pd.Series(votingC.predict(test), name="Survived")

results = pd.concat([IDtest,test_Survived],axis=1)
print(results)
results.to_csv("ensemble_python_voting.csv",index=False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


If you found this notebook helpful or you just liked it , some upvotes would be very much appreciated - That will keep me motivated :)